## Correlation of NKX2-5 ASE effect size with heart-specific eQTLs
We have observed that there is no correlation between ASE in NKX2-5 and gene expression levels of the nearest gene in iPSC-CMs, however there is an enrichment for heart-specific eQTLS in NKX2-5 ASE. Test the same but restricted to heart or heart-specific eQTLs - gene pairs.

In [1]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(gplots))

home     ='/home/paola/Family1070/private_output'
setwd(home)
ase_dir  = "ASE_chip/correlationWithBeta/"
gtex_dir = "Enrichment_annotations/GTEx/"

nm    = c('NKX25','H3K27AC_CM','H3K27AC_IPSC')
loci  = nm
QTL   = paste (nm, "_eQTLs.txt", sep="") 


corr         = paste(ase_dir , "Correlation_ase_snv_", loci, ".txt", sep="")
rna_cms      = 'PCA_rnaseq/iPSC_CM/residual_counts.txt'
chip_tables  = paste("PCA_chipseq", loci, 'residual_counts.txt',sep="/")
sample_table = read.csv("PCA_chipseq/fam1070_data_plus_production-1.csv", stringsAsFactors = FALSE) 


gene_info <- read.table("/publicdata/gencode_v19_20151104/gene_info.tsv", header=T, sep="\t", stringsAsFactor=F)

In [102]:
new.dir = "Coordination_ASE_effects/results/Coordination_at_heart_eQTLs"
dir.create(new.dir)

Warning message in dir.create(new.dir):
"'Coordination_ASE_effects/results/Coordination_at_heart_eQTLs' already exists"

In [103]:
scatterPlot_beta = function(ag, beta_x, beta_y,xlab, ylab, colors ="#1B69AF") {
    
ag = ag[order(ag$combined_fdr, decreasing=F),] 


xlim = c(-max(abs(ag[, beta_x])),max(abs(ag[, beta_x]))) 
ylim = c(-max(abs(ag[, beta_y])),max(abs(ag[, beta_y]))) 

plot(ag[, beta_x], ag[, beta_y], ylim=ylim, xlim=xlim,
     pch=16, cex=0.5, col=colors, 
     cex.lab=1.1, cex.axis=1.1, xlab = paste ("effect size", xlab),
     ylab = paste ("effect size", ylab), main=ylab)

abline(h=0,v=0, lty=2, col="red", lwd=1)

  
co = cor.test(ag[, beta_x], ag[, beta_y], method="spearman", exact=F)
l  = lm(ag[, beta_y]~ag[, beta_x] )
abline(l)
text (xlim[2]-xlim[2]/1.8, ylim[1]-ylim[1]/3
      , paste( "r=",round(co$estimate,2), "\n P=",signif(co$p.value,3) ),cex=1.1)

    return(c(co$estimate, co$p.value))
    
}

In [109]:
i = 2

In [110]:
eqtls = read.table( paste(gtex_dir, nm[i], ".GtexTable_hreg_aggregate.txt", sep=""), header=T,  sep="\t") 

heart_eqtls    = subset(eqtls, Hear>0)
heart_specific = subset(eqtls, ((eqtls[,2]/2)-eqtls[,"Hear"])<=0)
heart_eqtls    = merge( heart_eqtls [ ,c('snpID','gene')], gene_info[, c('gene_id','gene_name')], by.x ='gene', by.y='gene_name' )
heart_specific = merge( heart_specific [ ,c('snpID','gene')], gene_info[, c('gene_id','gene_name')], by.x ='gene', by.y='gene_name' )

ch = read.table(corr[i], header=TRUE, check.names=F)
rn        = read.table(rna_cms, header=T, check.names=F)
samp      = sample_table[sample_table$Cell_type == "iPSC-CM"& sample_table$Data_type=="RNA-Seq",]
closest   = "exp_iPSC_CM"
chp  = merge(ch, heart_eqtls, by.x="ID", by.y="snpID" )


chpr = merge(chp [c("gene", "gene_id","peakID" ,"varID" ,"combined_pv", "combined_fdr" ,"ref_freq", "Coefficient",
             "ID" ,"REF" ,"ALT",'iPSCORE_2_1', 'iPSCORE_2_2', 'iPSCORE_2_3' ,'iPSCORE_2_4', 'iPSCORE_2_6', 'iPSCORE_2_7' ,'iPSCORE_2_9')], 
                rn, by="gene_id", by.y="row.names")

peaks      = subset(chpr, select=as.character(samp$UUID))        
genotype   = subset(chpr, select=as.character(samp$Subject_ID))
   

res=data.frame()
for( m in 1:nrow(chpr)) {    
    
    df         = data.frame(peaks=t(peaks)[,m], genotype=t(genotype)[,m])
    df$subject = str_split_fixed(colnames(genotype), "\\.", 2)[,1]
    df$peaks   = scale(df$peak) # z-score normalized
    
    mod<-lm(peaks ~ genotype, data=df)  
    cof<-coef(summary(mod))[2,1]
    pv<-coef(summary(mod))[2,4]
    res[m,"Coefficient_closest"]<-cof
    res[m,"lm_pVal_closest"]<-pv
  
  }  


ag = cbind(chpr, res)
write.table(ag, paste( new.dir, "/Coordination_at_heart_eQTLs",  loci[i], "_with_", closest, ".txt", sep=""), 
            sep="\t", row.names=FALSE, col.names=TRUE)

    



pdf(paste(new.dir, "/Scatterplots_", nm[i], ".pdf", sep=""))

results = data.frame()
results["n","heart_eQTLs"] = nrow(ag)

par(mfrow=c(3,4), pin=c(1.3,1.3))

results[paste(c("r_","p_"), closest,nm[i], sep =""), 
        "heart_eQTLs"] = scatterPlot_beta(ag, 'Coefficient', 'Coefficient_closest',nm[i], closest, colors = "brown")

ag_specific = subset (ag, ID %in% as.character(heart_specific$snpID) )
results["n","heart_spec_eQTLs"] = nrow(ag_specific)

results[paste(c("r_","p_"), closest,nm[i], sep =""), 
        "heart_spec_eQTLs"] = scatterPlot_beta(ag_specific, 'Coefficient', 'Coefficient_closest',paste(nm[i], "heart spec."), closest)

plot.new()
plot.new()
h_tissues= c('Heart_Left_Ventricle_' , 'Heart_Atrial_Appendage_'  )

for (g in 1:2){

gtex_lv = read.table(paste("/publicdata/gtex_v6/", h_tissues[g], "Analysis.snpgenes",sep=""), header=T, sep="\t")

gtex_lv$varID = paste("chr", gtex_lv$snp_chrom, ":", gtex_lv$snp_pos, sep="")

gtex_lv = subset(gtex_lv, varID %in% ag$varID)

ag2 = merge( ag, gtex_lv [ ,c('beta', 'gene' ,'varID')], by.x=c("varID", 'gene_id'), by.y=c("varID", 'gene'))

results[paste(c("r_","p_"), h_tissues[g],nm[i], sep =""), 
        "heart_eQTLs"] = scatterPlot_beta(ag2, 'Coefficient','beta', nm[i],   h_tissues[g], colors = "brown")
results[paste(c("r_","p_"), h_tissues[g],closest, sep =""),
        "heart_eQTLs"] = scatterPlot_beta(ag2,  'Coefficient_closest', 'beta', closest, h_tissues[g], colors = "brown")
   
ag2_specific = merge( ag_specific, gtex_lv [ ,c('beta', 'gene' ,'varID')], by.x=c("varID", 'gene_id'), by.y=c("varID", 'gene'))

results[paste(c("r_","p_"), h_tissues[g],nm[i], sep =""), 
        "heart_spec_eQTLs"] = scatterPlot_beta(ag2_specific, 'Coefficient' ,'beta', paste(nm[i], "heart spec."),h_tissues[g])
results[paste(c("r_","p_"), h_tissues[g],closest, sep =""),
        "heart_spec_eQTLs"] = scatterPlot_beta(ag2_specific,'Coefficient_closest', 'beta', paste(closest, "heart spec."), h_tissues[g])
   
 
}
dev.off()

rho  = results[c(FALSE,TRUE), ]
pval = results[c(TRUE,FALSE), ]
pval = -log( pval [-1,],10)

png 
  2

In [111]:
pdf(paste(new.dir, "/Heatmap_", nm[i], ".pdf", sep=""))
my_palette = colorRampPalette(c( "white","dodgerblue3"))(10)
heatmap.2(as.matrix(pval) , srtRow=0, srtCol=45, offsetRow=-0.5, offsetCol=-0.5, 
          keysize=1, margins =c(28,29), trace="none",Colv=F,Rowv=F,
          key.title="-Log10(P)", cellnote=round(rho,2), notecol="black", 
          cexRow=1, cexCol=1, col=my_palette,
          sepwidth=c(0.01,0.01),sepcolor="black",colsep=0:ncol(pval),rowsep=0:nrow(pval))
dev.off()

Warning message in heatmap.2(as.matrix(pval), srtRow = 0, srtCol = 45, offsetRow = -0.5, :
"Discrepancy: Rowv is FALSE, while dendrogram is `both'. Omitting row dendogram."Warning message in heatmap.2(as.matrix(pval), srtRow = 0, srtCol = 45, offsetRow = -0.5, :
"Discrepancy: Colv is FALSE, while dendrogram is `column'. Omitting column dendogram."

png 
  2

In conclusion we observed that there is a a good correlation between the eQTLS from gtex ang the effects observed in iPSC-CMs at these loci. Possible implication of NKX2-5 binding

### GO analysis on genes that have ASE in NKX2-5 and that have GTEx eQTL in heart (n=116) or heart-specific (n=39)

In [88]:
suppressPackageStartupMessages(library(goseq))

In [112]:
dir.create(paste( new.dir, "GO", sep="/"))
pdf(paste(new.dir, "/GO/", nm[i], "_GO.pdf", sep=""))
    par(mfrow=c(2,2), mar=c(2,15,2,1))    
   
for (t in 1:3){
   
if(t==1){    
    set = "heart_eqtls"
    test        = rownames(rn) %in% ag$gene_id
    names(test) = str_split_fixed(rownames(rn),"\\.",2)[,1]
    test = test[!duplicated(names(test))]
    }
if(t==2){    
    set = "heart_specific_eqtls"
    test        = rownames(rn) %in% ag_specific$gene_id
    names(test) = str_split_fixed(rownames(rn),"\\.",2)[,1]
    test = test[!duplicated(names(test))]
    }
if(t==3){    
    set = "heart_specific_versus_heart_ eqtls"
    test        = ag$gene_id %in% ag_specific$gene_id
    names(test) = str_split_fixed(ag$gene_id,"\\.",2)[,1]
    test        = test[!duplicated(names(test))]
}

    pwf         = nullp(test,"hg19","ensGene", plot=F)
    GO          = goseq(pwf,"hg19","ensGene", test.cats=c("GO:BP"))
    GO$Bonferroni  = -log(p.adjust(GO$over_represented_pvalue, method="bonferroni"),10)
    GO             = GO[order(GO$over_represented_pvalue, decreasing=F),]
      
barplot(GO$Bonferroni[20:1], names.arg=GO$term[20:1], horiz=T, main=set,
    las=1, cex.axis=0.8, cex.names=0.8, density=c(-1, 20)[(GO$Bonferroni[20:1]<=1.3)+1])
    abline(v=1.3, lwd=2, lty=2, col="red")      
          
}
dev.off()

Warning message in dir.create(paste(new.dir, "GO", sep = "/")):
"'Coordination_ASE_effects/results/Coordination_at_heart_eQTLs/GO' already exists"Loading hg19 length data...
Fetching GO annotations...
For 2822 genes, we could not find any categories. These genes will be excluded.
To force their use, please run with use_genes_without_cat=TRUE (see documentation).
This was the default behavior for version 1.15.1 and earlier.
Calculating the p-values...
'select()' returned 1:1 mapping between keys and columns
Loading hg19 length data...
Warning message in pcls(G):
"initial point very close to some inequality constraints"Fetching GO annotations...
For 2822 genes, we could not find any categories. These genes will be excluded.
To force their use, please run with use_genes_without_cat=TRUE (see documentation).
This was the default behavior for version 1.15.1 and earlier.
Calculating the p-values...
'select()' returned 1:1 mapping between keys and columns
Loading hg19 length data...
Warning m

png 
  2

No enrichment for GO term was found